# EXP 14. Movielens 영화 추천

## 0. 들어가며

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.



### 0.1 Import package

In [1]:
# 커널정보 3.7
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 1. 데이터 준비와 전처리
***
Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
rating_file_path = 'D:/project/aiffel_exp/exp14_recommand system/data/project/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::',
                      names=ratings_cols, engine='python', encoding="ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='D:/project/aiffel_exp/exp14_recommand system/data/project/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [7]:
# movies와 ratings을 합침
data = pd.merge(movies, ratings)
cols = cols + ratings_cols 
data.head()

,movie_id,title,genre,user_id,counts,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [8]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['movie_id', 'title', 'user_id', 'counts']
data = data[using_cols]
data.head()

,movie_id,title,user_id,counts
0,1,Toy Story (1995),1,5
1,1,Toy Story (1995),6,4
2,1,Toy Story (1995),8,4
3,1,Toy Story (1995),9,5
4,1,Toy Story (1995),10,5


In [9]:
# 검색을 쉽게 하기 위해 타이틀 문자열을 소문자로 바꿔줍시다.
data['title'] = data['title'].str.lower() 
data.head()

,movie_id,title,user_id,counts
0,1,toy story (1995),1,5
1,1,toy story (1995),6,4
2,1,toy story (1995),8,4
3,1,toy story (1995),9,5
4,1,toy story (1995),10,5


In [10]:
# 첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 봅시다.
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,movie_id,title,user_id,counts
0,1,toy story (1995),1,5
19773,48,pocahontas (1995),1,5
35123,150,apollo 13 (1995),1,5
54895,260,star wars: episode iv - a new hope (1977),1,4
116019,527,schindler's list (1993),1,5
119008,531,"secret garden, the (1993)",1,4
130190,588,aladdin (1992),1,4
138999,594,snow white and the seven dwarfs (1937),1,4
139706,595,beauty and the beast (1991),1,5
142605,608,fargo (1996),1,4


## 2. 분석해 봅시다.
***
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [11]:
# 유니크한 영화 수
data['movie_id'].nunique()

3628

In [12]:
# 유니크한 유저 수
data['user_id'].nunique()

6039

In [13]:
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(20)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [14]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

수치에 대한 의문, 1968편의 영화를 보는게 가능한가..?

In [15]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

## 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [16]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['matrix, the (1999)',
               'jurassic park (1993)',
               'sixth sense, the (1999)',
               'fargo (1996)',
               'braveheart (1995)']

# 'jby'이라는 user_id가 위 영화를 평점 5점씩 줬다고 가정
my_playlist = pd.DataFrame({'user_id': ['jby']*5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['jby']})['user_id'].any():  # user_id에 'jby'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,movie_id,title,user_id,counts
836473,3952.0,"contender, the (2000)",5682,3
836474,3952.0,"contender, the (2000)",5812,4
836475,3952.0,"contender, the (2000)",5831,3
836476,3952.0,"contender, the (2000)",5837,4
836477,3952.0,"contender, the (2000)",5998,4
0,NaN,"matrix, the (1999)",jby,5
1,NaN,jurassic park (1993),jby,5
2,NaN,"sixth sense, the (1999)",jby,5
3,NaN,fargo (1996),jby,5
4,NaN,braveheart (1995),jby,5


In [17]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [18]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['jby'])    # 6038명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 합니다. 
print(title_to_idx['matrix, the (1999)'])

6039
2325


In [20]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing Fail!!
title column indexing OK!!


,movie_id,title,user_id,counts
0,1.0,0,0,5
1,1.0,0,1,4
2,1.0,0,2,4
3,1.0,0,3,5
4,1.0,0,4,5
...,...,...,...,...
0,NaN,2325,6039,5
1,NaN,462,6039,5
2,NaN,2507,6039,5
3,NaN,587,6039,5


## 4. CSR matrix를 직접 만들어 봅시다.

In [24]:
# 실습 위에 설명보고 이해해서 만들어보기
num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [25]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [32]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4227959674179612400,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15476830468957888627
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5214918221600458737
 physical_device_desc: "device: XLA_GPU device"]

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
FACTORS = 100
ITTER = 30

als_model = AlternatingLeastSquares(factors=FACTORS,
                                    regularization=0.01,
                                    use_gpu=True,
                                    iterations=ITTER,
                                    dtype=np.float32)

ValueError: No CUDA extension has been built, can't train on GPU.

In [31]:
import nvidia
nvidia-smi

ModuleNotFoundError: No module named 'nvidia'

In [28]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.